In [2]:
from oic.oic import Client
from oic.utils.authn.client import CLIENT_AUTHN_METHOD
from oic.oic.message import ProviderConfigurationResponse, RegistrationResponse, AuthorizationResponse, ClaimsRequest, Claims
from oic import rndstr

client = Client(client_authn_method=CLIENT_AUTHN_METHOD)

client_id = "uadventure"
client_secret = ""
info = {"client_id": client_id, "client_secret": client_secret}

client_reg = RegistrationResponse(**info)
client.store_registration_info(client_reg)
issuer= 'https://sso.simva.e-ucm.es/auth/realms/simva/protocol/openid-connect/'
authorization_base_url = 'https://sso.simva.e-ucm.es/auth/realms/simva/protocol/openid-connect/auth'
token_url = 'https://sso.simva.e-ucm.es/auth/realms/simva/protocol/openid-connect/token'
op_info = ProviderConfigurationResponse(
    version="1.0", issuer=issuer,
    authorization_endpoint=authorization_base_url,
    token_endpoint=token_url)

client.handle_provider_config(op_info, op_info['issuer'])

redirect_uri='https://api.simva.e-ucm.es/activities/5f6214c0e195220016c87124/result'

session={"state": rndstr(), "nonce": rndstr()}


claims_request=ClaimsRequest(
    id_token=Claims(username=True),
    userinfo=Claims(name=None))

args_request = {
    "client_id": client.client_id,
    "response_type": "code",
    "scope": ["openid"],
    "nonce": session["nonce"],
    "redirect_uri": redirect_uri,
    "state": session["state"],
    "claims":claims_request
}

auth_req = client.construct_AuthorizationRequest(request_args=args_request)
login_url = auth_req.request(client.authorization_endpoint)+"&token"

print(auth_req)
print(login_url)

#fill form
%run browserFormFiler.ipynb
login_url_form_fill=fill_form(login_url, 'pg4d')

print(login_url_form_fill)
print(client)

#code given
aresp = client.parse_response(AuthorizationResponse, info=login_url_form_fill, sformat="urlencoded")
code = aresp["code"]
assert aresp["state"] == session["state"]
print(code)

args = {
    "code": code,
    "client_id":client.client_id,
    "redirect_uri":redirect_uri
}
resp = client.do_access_token_request(state=aresp["state"],
                                      request_args=args,
                                      authn_method="client_secret_basic")
print(resp)

userinfo = client.do_user_info_request(state=aresp["state"])
print(userinfo)

{'client_id': 'uadventure', 'response_type': 'code', 'scope': 'openid', 'nonce': '9EO2VPmUhmlwsmhr', 'redirect_uri': 'https://api.simva.e-ucm.es/activities/5f6214c0e195220016c87124/result', 'state': '1wltn5wWJ97Tvwb4', 'claims': {'id_token': {'username': True}, 'userinfo': {'name': None}}}
https://sso.simva.e-ucm.es/auth/realms/simva/protocol/openid-connect/auth?client_id=uadventure&response_type=code&scope=openid&nonce=9EO2VPmUhmlwsmhr&redirect_uri=https%3A%2F%2Fapi.simva.e-ucm.es%2Factivities%2F5f6214c0e195220016c87124%2Fresult&state=1wltn5wWJ97Tvwb4&claims=%7B%22id_token%22%3A+%7B%22username%22%3A+true%7D%2C+%22userinfo%22%3A+%7B%22name%22%3A+null%7D%7D&token


AttributeError: 'NoneType' object has no attribute 'find_element_by_xpath'

In [ ]:
import requests

urlpost="https://api.simva.e-ucm.es/activities/5f6214c0e195220016c87124/result?state=BzjCQg96TbLaYUEs&session_state=90894e2e-eff9-4bd5-8c4e-585494822fa3&code=5a4a5b0a-1d7c-4662-9eeb-4ed60b498df0.90894e2e-eff9-4bd5-8c4e-585494822fa3.edfc1ac7-f2cf-4789-80b3-4ad4c1e75290"
def post_activity_xapi_data(dataxapilist):
    for dataxapi in dataxapilist:
        response=requests.post(urlpost, data=dataxapi)#redirect.message
        print(response.status_code)
        print(response.content)

data=[
  {
    "actor": {
      "id": "63ba693f-23f5-41ef-8660-bdc00adc1e5f",
      "name": "63ba693f-23f5-41ef-8660-bdc00adc1e5f"
    },
    "verb": {
      "id": "https://w3id.org/xapi/seriousgames/verbs/interacted"
    },
    "object": {
      "id": "13651f64-0fb7-4e46-aacd-4069c94c0dcd",
      "definition": {
        "type": "https://w3id.org/xapi/seriousgames/activity-types/enemy"
      }
    },
    "timestamp": "2020-09-16T16:42:23.095536Z"
  },
  {
    "actor": {
      "id": "488de066-18b0-4e10-bd29-a38b2a82ea33",
      "name": "488de066-18b0-4e10-bd29-a38b2a82ea33"
    },
    "verb": {
      "id": "https://w3id.org/xapi/seriousgames/verbs/selected"
    },
    "object": {
      "id": "47290d8e-5f82-4e55-be9d-db0ce51734b9",
      "definition": {
        "type": "https://w3id.org/xapi/seriousgames/activity-types/question"
      }
    },
    "timestamp": "2020-09-16T16:42:42.844953Z"
  }
]
post_activity_xapi_data(data)